# Make bookkeeping with Holograms


- work with Weakly_2022_09
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/20


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/atmospec 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_09/conda/envs/lsst-scipipe-2.0.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/rapid_analysis 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g64fc59b30a+e1dba0a6f6 	w_latest w_2022_09 current setup


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [4]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [5]:
import lsst.daf.butler as dafButler

In [6]:
repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

In [7]:
collection = "u/dagoret/first_test_autxel"
datasetRefs1 = registry.queryDatasets(datasetType='postISRCCD', collections=collection, where= "instrument='LATISS'")
datasetRefs2 = registry.queryDatasets(datasetType='calexp', collections=collection, where= "instrument='LATISS'")

## List of Exposures

In [8]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs','group_name' ,'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','postISRCCD','calexp'])

In [9]:
df_exposure.astype({"id":str,
                    "obs_id":str,
                    "day_obs": str,
                    "group_name":str,
                    "seq_num":'int32',
                    'type':str,
                    "target":str,
                    "filter":str,
                    "zenith_angle":'float',
                    "expos":'float',
                    "ra":'float',
                    "dec":'float',
                    "skyangle":'float',
                    "postISRCCD":bool,
                    "calexp":bool
                   }              
                  )

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp


In [10]:
#df_exposure = pd.DataFrame( {"id":'int64',
#                             "obs_id":str,
#                             "day_obs": str,
#                             "seq_num":'int32',
#                             "type":str,
#                             "target":str,
#                             "filter":str,
#                             "zenith_angle":'float32',
#                             "expos":'float32',
#                             "ra":'float32',
#                             "dec":'float32',
#                             "skyangle":'float32'})                  

In [11]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    flag_postisrccd = False
    flag_calexp = False
    
    for count, ref in enumerate(datasetRefs1):    
        if ref.dataId["exposure"]== info.id:
            flag_postisrccd = True
            break
    for count, ref in enumerate(datasetRefs2):    
        if ref.dataId["exposure"]== info.id:
            flag_calexp = True
            break        
            
    
    
    
    df_exposure.loc[count] = [str(info.id), str(info.obs_id), str(info.day_obs), str(info.group_name),int(info.seq_num), str(info.observation_type), str(info.target_name), str(info.physical_filter), info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,flag_postisrccd,flag_calexp]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t group_name:          ",info.group_name)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [12]:
df_exposure.dtypes

id               object
obs_id           object
day_obs          object
group_name       object
seq_num           int64
type             object
target           object
filter           object
zenith_angle    float64
expos           float64
ra              float64
dec             float64
skyangle        float64
postISRCCD         bool
calexp             bool
dtype: object

In [13]:
df_exposure

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp
2669,2022063000805,AT_O_20220630_000805,20220630,2022-07-01T09:25:58.670,805,science,AUXTEL_DRP_IMAGING:WFI-2026-4536_064,SDSSr~empty,36.666615,30.0,306.944896,-45.325220,359.998972,False,False
97,2021070700320,AT_O_20210707_000320,20210707,2021-07-08T04:49:21.820,320,science,HD 160617,empty~holo4_003,20.545791,20.0,265.639181,-40.262824,-1.402388,True,False
95,2021070700319,AT_O_20210707_000319,20210707,2021-07-08T04:49:21.820,319,science,HD 160617,empty~holo4_003,20.454183,20.0,265.636042,-40.264335,-1.682716,True,False
99,2021070700318,AT_O_20210707_000318,20210707,2021-07-08T04:49:21.820,318,science,HD 160617,empty~holo4_003,20.362633,20.0,265.632998,-40.265829,-1.964402,True,False
98,2021070700321,AT_O_20210707_000321,20210707,2021-07-08T04:49:21.820,321,science,HD 160617,empty~holo4_003,20.640626,20.0,265.642182,-40.261426,-1.114358,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1922,2022063000590,AT_O_20220630_000590,20220630,2022-07-01T06:23:28.830,590,science,spec:HD185975,empty~holo4_003,57.082485,30.0,306.150728,-87.470936,268.302522,True,False
2368,2022063000591,AT_O_20220630_000591,20220630,2022-07-01T06:23:28.830,591,science,spec:HD185975,BG40~holo4_003,57.082091,30.0,306.150729,-87.470937,268.302294,True,False
2374,2022063000592,AT_O_20220630_000592,20220630,2022-07-01T06:23:28.830,592,science,spec:HD185975,BG40~holo4_003,57.081747,30.0,306.150722,-87.470936,268.302276,True,False
1423,2022063000593,AT_O_20220630_000593,20220630,2022-07-01T06:23:28.830,593,science,spec:HD185975,FELH0600~holo4_003,57.081424,30.0,306.150684,-87.470935,268.302255,True,False


## Selection of science

In [14]:
df_science = df_exposure[df_exposure.type == 'science']

In [15]:
df_science.sort_values(by="id",ascending=True,inplace=True)

In [16]:
df_science.tail(60)

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp
2189,2022063000393,AT_O_20220630_000393,20220630,2022-07-01T03:49:47.379,393,science,spec:HD142331,BG40~holo4_003,36.003351,30.0,238.612411,-8.549413,41.858147,True,False
2627,2022063000394,AT_O_20220630_000394,20220630,2022-07-01T03:49:47.379,394,science,spec:HD142331,FELH0600~holo4_003,36.140515,30.0,238.612407,-8.549382,41.858228,True,False
2614,2022063000395,AT_O_20220630_000395,20220630,2022-07-01T03:49:47.379,395,science,spec:HD142331,FELH0600~holo4_003,36.274797,30.0,238.612431,-8.549375,41.858224,True,False
1794,2022063000405,AT_O_20220630_000405,20220630,2022-07-01T04:00:56.082,405,science,spec:HD142331,empty~holo4_003,37.445817,30.0,238.611319,-8.547952,39.700941,True,False
1764,2022063000406,AT_O_20220630_000406,20220630,2022-07-01T04:00:56.082,406,science,spec:HD142331,empty~holo4_003,37.582514,30.0,238.611305,-8.548004,39.700817,True,False
2195,2022063000407,AT_O_20220630_000407,20220630,2022-07-01T04:00:56.082,407,science,spec:HD142331,BG40~holo4_003,37.729001,30.0,238.611328,-8.548027,39.700787,True,False
2239,2022063000408,AT_O_20220630_000408,20220630,2022-07-01T04:00:56.082,408,science,spec:HD142331,BG40~holo4_003,37.866224,30.0,238.611327,-8.548002,39.700848,True,False
2663,2022063000409,AT_O_20220630_000409,20220630,2022-07-01T04:00:56.082,409,science,spec:HD142331,FELH0600~holo4_003,38.010065,30.0,238.611351,-8.547972,39.700889,True,False
2641,2022063000410,AT_O_20220630_000410,20220630,2022-07-01T04:00:56.082,410,science,spec:HD142331,FELH0600~holo4_003,38.147741,30.0,238.611343,-8.547981,39.700915,True,False
1712,2022063000420,AT_O_20220630_000420,20220630,2022-07-01T04:12:49.301,420,science,spec:HD185975,empty~holo4_003,57.565624,30.0,306.203302,-87.495714,236.053076,True,False


# Generate Book Keeping files

In [17]:
writer = pd.ExcelWriter('BookKeepingAuxtelHologram_raw.xlsx', engine='xlsxwriter')
current_page = 0

In [18]:
ListOfDates = df_science.day_obs.unique()

In [19]:
ListOfDates

array(['20210707', '20210909', '20211102', '20211103', '20211104',
       '20220215', '20220216', '20220316', '20220317', '20220607',
       '20220608', '20220609', '20220628', '20220629', '20220630'],
      dtype=object)

In [20]:
for date_sel in ListOfDates:
    
    # get for that date
    df_science_selected = df_science[df_science.day_obs == date_sel ]
    N = len(df_science_selected)
    
    list_of_filters = df_science_selected['filter'].unique()
    
    selected_filters = []
    # Select hologram
    for filt in list_of_filters:
        #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
        flag_sel = (filt.find('holo4') != -1) 
        #flag_sel = (filt.find('ronchi170') != -1) 
        if flag_sel:
            selected_filters.append(filt) 
    Nfilt = len(selected_filters)
    # sort by filter name
    if(Nfilt>0):
        selected_filters = np.array(selected_filters)
        df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
        df_science_selected.to_excel(writer, sheet_name=str(date_sel))
        


/tmp/ipykernel_1359/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
/tmp/ipykernel_1359/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
/tmp/ipykernel_1359/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_va

In [21]:
writer.save()